In [1]:
# For Installation (Required)
!pip install pyLDAvis
!pip install glove-python-binary

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import re
import string
import nltk
import gensim
import time
import pickle
import warnings
warnings.filterwarnings('ignore')

import pyLDAvis
import pyLDAvis.gensim_models as gensimvis

from gensim import corpora,models
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from gensim.models import CoherenceModel
from gensim.models import Word2Vec,FastText
from gensim.test.utils import get_tmpfile

from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

from glove import Glove
from glove import Corpus

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.decomposition import PCA

from scipy.spatial import distance

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


In [3]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [4]:
#Import Whole Dataset
firefox_data=pd.read_csv('/content/drive/Shareddrives/DSCI-644-Team-5/PrimaryDataset/firefox_preproccessed_whole_dataset.csv')

#Cleaning the Data
firefox_data = firefox_data.reset_index()
firefox_data = firefox_data.drop(columns = ["index","Unnamed: 0"])
firefox_data = firefox_data.drop_duplicates()
firefox_data = firefox_data.dropna(axis=0, subset=['Bug ID'])

#To show Data
firefox_data.head()

,Bug ID,Type,Summary,Product,Component,Status,Resolution,Updated,Description,Duplicate_Bug_Ids
0,1399936,defect,Shield unenrolls users in active preference ex...,Firefox,Normandy Client,VERIFIED,FIXED,2018-01-27 11:10:40,the activity stream team is conducting a study...,NaN
1,1387208,defect,browser_as_load_location.js is permafailing on...,Firefox,New Tab Page,RESOLVED,FIXED,2019-06-28 14:29:46,filed by rvandermeulen at mozilla com j...,NaN
2,1368432,defect,web.skype.com is inaccessible: SEC_ERROR_OCSP_...,Firefox,Untriaged,RESOLVED,DUPLICATE,2017-05-29 04:49:00,user agent mozilla gecko firefox ...,1368433.0
3,1383741,defect,Perma failure when 56 merge to beta in browser...,Firefox,Preferences,RESOLVED,FIXED,2017-07-27 02:02:17,found via uplift simulations perma failure w...,NaN
4,1388753,defect,Bookmarks and history pages in Library sub men...,Firefox,Toolbars and Customization,VERIFIED,FIXED,2017-09-22 10:09:03,user agent mozilla gecko firefox ...,NaN


In [5]:
#To show Info
firefox_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 38290 entries, 0 to 38289
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Bug ID             38290 non-null  int64  
 1   Type               38290 non-null  object 
 2   Summary            38290 non-null  object 
 3   Product            38290 non-null  object 
 4   Component          38290 non-null  object 
 5   Status             38290 non-null  object 
 6   Resolution         38290 non-null  object 
 7   Updated            38290 non-null  object 
 8   Description        38290 non-null  object 
 9   Duplicate_Bug_Ids  7859 non-null   float64
dtypes: float64(1), int64(1), object(8)
memory usage: 3.2+ MB


# **Cleaning and Preprocessing**

In [6]:
#Text Cleaning to Remove Punctuations
def clean_text_round_1(text):
    text = re.sub(r'\w*\d\w*', '', text)
    text = re.sub(r'\w*\f\w*', '', text)
    text = re.sub(r'\(.*?\)', '', text)
    text = re.sub(r'\[.*]\)', '', text)
    text = text.lower()
    text = re.sub(r'[%s]' % re.escape(string.punctuation), '', text)
    return text

round1 = lambda x: clean_text_round_1(x)

#Text Cleaning to Remove Additional Punctuations
def clean_text_round_2(text):
    text = re.sub(r'[‘’“”…]', '', text)
    text = re.sub(r'\n', '', text)
    text = re.sub(r'\t', '', text)
    return text

round2 = lambda x: clean_text_round_2(x)

In [7]:
#To Clean Data in 'Description' Column
firefox_data["Description"]= firefox_data["Description"].str.replace("fixed in HEAD", "", case = False)
firefox_data["Description"]= firefox_data["Description"].str.replace("has been marked as readonly", " ", case = False)

firefox_data = firefox_data.dropna(axis=0, subset=['Description'])

firefox_data['Description'] = firefox_data['Description'].apply(clean_text_round_1)
firefox_data['Description'] = firefox_data['Description'].apply(clean_text_round_2)

#To show Info
firefox_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 38290 entries, 0 to 38289
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Bug ID             38290 non-null  int64  
 1   Type               38290 non-null  object 
 2   Summary            38290 non-null  object 
 3   Product            38290 non-null  object 
 4   Component          38290 non-null  object 
 5   Status             38290 non-null  object 
 6   Resolution         38290 non-null  object 
 7   Updated            38290 non-null  object 
 8   Description        38290 non-null  object 
 9   Duplicate_Bug_Ids  7859 non-null   float64
dtypes: float64(1), int64(1), object(8)
memory usage: 3.2+ MB


In [8]:
#Helper Functions for Preprocessing
def lemmatize(text):
    return WordNetLemmatizer().lemmatize(text, pos='v')

def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 5:
            result.append(lemmatize(token))
    return result

In [9]:
#To Preprocess Data in 'Description' Column
firefox_data['Description'] = firefox_data['Description'].map(preprocess)

print('Null Duplicate Bug Ids: ',firefox_data['Duplicate_Bug_Ids'].isnull().sum())

#To save Duplicate Reports in a CSV File
duplicate_reports = firefox_data.dropna(axis=0, subset=['Duplicate_Bug_Ids'])
duplicate_reports.reset_index(drop=True)
duplicate_reports.to_csv('firefox_duplicate_reports.csv')

#Seperating all the master reports into a dataframe
master_reports = firefox_data[firefox_data.isnull().any(axis=1)]
master_reports.reset_index(drop=True)

print('NA Values in Master Report: ', master_reports.Description.isna().sum())

#To save Master Reports in a CSV File
master_reports.to_csv('firefox_master_reports.csv')

Null Duplicate Bug Ids:  30431
NA Values in Master Report:  0


In [10]:
#Import Master Reports
master_reports = pd.read_csv('firefox_master_reports.csv')
master_reports = master_reports.drop(columns=['Unnamed: 0'])

#To Preprocess Data in 'Description' Column
master_reports['Description'] = master_reports['Description'].map(preprocess)

In [11]:
#To Create a Dictionary
dictionary = gensim.corpora.Dictionary(master_reports['Description'])
dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)

#To Create BoW a Dictionary
bow_corpus = [dictionary.doc2bow(doc) for doc in master_reports['Description']]

In [12]:
#To Open Pickle File
file_bow = open('firefox_bow_corpus.pickle', 'wb')

#To dump BoW data in Pickle File
pickle.dump(bow_corpus, file_bow)

#To Open Pickle File
file_dict = open('firefox_dictionary.pickle', 'wb')

#To dump Dictionary data in Pickle File
pickle.dump(dictionary, file_dict)

In [1]:
#Parameters for LDA Model
corpus = bow_corpus
no_of_topics = 10
dictionary = dictionary
p = 20
k = 2
epochs = 100

#Training the LDA model on the BoW corpus
lda_model = gensim.models.LdaMulticore(corpus, num_topics=no_of_topics, id2word=dictionary, passes=p, workers=k, iterations=epochs)

In [14]:
#Saving the Model
lda_model.save('firefox_lda_model.model')

/usr/local/lib/python3.8/dist-packages/smart_open/smart_open_lib.py:496: DeprecationWarning: This function is deprecated.  See https://github.com/RaRe-Technologies/smart_open/blob/develop/MIGRATING_FROM_OLDER_VERSIONS.rst for more information
  warnings.warn(message, category=DeprecationWarning)
/usr/local/lib/python3.8/dist-packages/smart_open/smart_open_lib.py:496: DeprecationWarning: This function is deprecated.  See https://github.com/RaRe-Technologies/smart_open/blob/develop/MIGRATING_FROM_OLDER_VERSIONS.rst for more information
  warnings.warn(message, category=DeprecationWarning)
/usr/local/lib/python3.8/dist-packages/smart_open/smart_open_lib.py:496: DeprecationWarning: This function is deprecated.  See https://github.com/RaRe-Technologies/smart_open/blob/develop/MIGRATING_FROM_OLDER_VERSIONS.rst for more information
  warnings.warn(message, category=DeprecationWarning)


In [2]:
#Evaluation of Model

#Perplexity
print('Perplexity: ', lda_model.log_perplexity(bow_corpus))  

#Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=firefox_data['Description'], dictionary=dictionary, coherence='c_v')
print('\nCoherence Score: ', coherence_model_lda.get_coherence())

Perplexity:  -5.485928713312202

Coherence Score:  0.5390895826383824


In [16]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, bow_corpus, dictionary)

In [17]:
#Visualization
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
6      0.192609 -0.179711       1        1  27.765300
4     -0.208116 -0.073758       2        1  16.293659
3     -0.247777 -0.089970       3        1  10.582644
8      0.230086 -0.172904       4        1  10.007827
7     -0.187826  0.022739       5        1   8.118777
1     -0.118048 -0.065253       6        1   6.458552
2      0.107400  0.153176       7        1   6.447605
5      0.266739  0.085332       8        1   6.065415
9     -0.016631  0.008600       9        1   4.873069
0     -0.018436  0.311748      10        1   3.387154, topic_info=           Term          Freq         Total Category  logprob  loglift
31      browser  89363.000000  89363.000000  Default  30.0000  30.0000
135    resource  13076.000000  13076.000000  Default  29.0000  29.0000
868      thread  14820.000000  14820.000000  Default  28.0000  28.0000
44      firefox  30056.000000  30056.000000  Default  27.0000  27.0000
52       result  25813.000000  25813.000000  Default  26.0000  26.0000
..          ...           ...           ...      ...      ...      ...
44      firefox    732.716850  30056.069110  Topic10  -4.0502  -0.3289
65      request    344.920412   2094.592904  Topic10  -4.8037   1.5814
160     message    412.786950  15011.591488  Topic10  -4.6240  -0.2085
33   extensions    348.864514   2694.537055  Topic10  -4.7923   1.3409
320     central    344.218539   3471.509217  Topic10  -4.8057   1.0741

[708 rows x 6 columns], token_table=      Topic      Freq            Term
term                                 
2095      1  0.995352  aboutcerterror
2647      1  0.302215     aboutlogins
2647      4  0.001467     aboutlogins
2647      8  0.692454     aboutlogins
2647     10  0.004401     aboutlogins
...     ...       ...             ...
448       2  0.915930         youtube
448       3  0.074690         youtube
448       9  0.009828         youtube
994       1  0.931473     zombiecheck
994       4  0.068263     zombiecheck

[1541 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[7, 5, 4, 9, 8, 2, 3, 6, 10, 1])

In [18]:
#Creating 10 empty clusters and pushing master reports in each of them based on topic modeling and saving them in individual csv file
for c in range(10):
    exec('topic_{} = pd.DataFrame()'.format(c))
    for i in range(len(master_reports)):
        topic=lda_model[dictionary.doc2bow(master_reports.Description[i])]
        topic= np.asarray(topic)
        if int(topic[np.argmax(topic[:,1]),0])== c:
            exec('topic_{} = topic_{}.append(master_reports.loc[[i]])'.format(c,c))
            exec('topic_{} = topic_{}.reset_index(drop=True)'.format(c,c))
            exec('topic_{}.to_csv("topic_{}.csv")'.format(c,c))

In [2]:
#To Open Pickle File
file_bow = open('firefox_bow_corpus.pickle', 'rb')
bow_corpus = pickle.load(file_bow)

file_dict = open('firefox_dictionary.pickle', 'rb')
dictionary=pickle.load(file_dict)

#To load Trained Model
lda_model =  models.LdaModel.load('firefox_lda_model.model')

In [20]:
#To Create corpus for Word2Vec and FastText models
for i in range(10):
    exec('sent_{} = []'.format(i))
    exec('x= topic_{}'.format(i))
    for j in range(len(x)):
        exec('sent_{}.append(topic_{}.Description[{}])'.format(i,i,j))

for sent in range(10):
    exec('print(len(sent_{}))'.format(sent))

1212
3148
1065
5222
9753
1141
2895
5168
514
320


In [2]:
#Training GloVe model for each cluster
for cluster in range(10):
    vector_size = 100
    exec('glove_corpus{}=Corpus()'.format(cluster, cluster)) 
    exec('glove_corpus{}.fit(sent_{})'.format(cluster, cluster))
    exec('glove{}= Glove(no_components=vector_size, learning_rate=0.18, alpha=0.75, max_count=100, max_loss=10.0, random_state=None)'.format(cluster, cluster))
    exec('glove{}.fit(glove_corpus{}.matrix, epochs=200, no_threads=3, verbose=True)'.format(cluster, cluster))
    exec('transformer = lambda dictionary2:glove{}.transform_paragraph(words, epochs=1000,ignore_missing=False)'.format(cluster, cluster))
    exec('glove{}.add_dictionary(glove_corpus{}.dictionary)'.format(cluster, cluster))

    #Save the all the models in individual file
    exec('path = get_tmpfile("glove{}.model")'.format(cluster))
    exec('glove{}.save("glove{}.model")'.format(cluster, cluster))

# **Classification**

In [22]:
#To Import all the Clusters created using LDA based Topic Modeling
for c in range(10):
    exec('topic_{} = pd.read_csv("topic_{}.csv")'.format(c,c))
    exec("topic_{}= topic_{}.drop(columns=['Unnamed: 0'])".format(c,c))
    exec("topic_{}['Description'] = topic_{}['Description'].map(preprocess)".format(c,c))

In [23]:
#To Import all the trained GloVe models
for mod in range(10):  
    exec('glove{} = Glove.load("glove{}.model")'.format(mod, mod))

In [24]:
#This will return the index of cluster in which the master report of duplicate report may reside
def sim_with_clusters_lda_topn(DR, n):
    vec_bow = dictionary.doc2bow(DR)
    x= lda_model[vec_bow]
    topic = np.asarray(x)
    sim=[]
    x= topic[np.argsort(topic[:,1])[-n:][::-1],0]

    for i in range(len(x)):
        sim.append(int(x[i]))

    return sim

#To get Similarity between two feature vectors using the average of Cosine Similarity & Euclidean Similarity
def sim(vec1, vec2): 
    sim1 = 1/(1+np.linalg.norm(np.array(vec1) - np.array(vec2)))
    sim2 = cosine_similarity(vec1, vec2)
    sim=(sim1+sim2)/2 
    return sim

# **LDA and GLOVE**

In [25]:
#Returns Top-N Master Reports
def compare_topn(model, cluster, sent, DR, topn, modal):
    similarity=[]
    
    if (modal == 'single'):
        vec_duplicate, master= feature_vectors_single_modality(DR, sent, model)
    else:
        raise ValueError('Invalid Modality entered')

    for doc in range(len(master)):
        vec_master = master[doc]
        vec_master= [vec_master]
        unified_sim = sim(vec_duplicate, vec_master)
        similarity.append(unified_sim)
    
    similarity = np.asarray(similarity)
    similarity= np.concatenate(similarity, axis=0 )
    similarity= np.concatenate(similarity, axis=0 )
    max_similar_reports=similarity.argsort()[-topn:][::-1]

    return(max_similar_reports)

In [26]:
# creation of feature vectors by singlemodality feature extraction
def feature_vectors_single_modality(DR, corpus, model):
    master = averaged_word_vectorizer_glove(corpus=sent, model=model, num_features=100)

    vec_duplicate = averaged_word_vectorizer_glove(corpus=DR, model=model, num_features=100)

    vec_duplicate = [vec_duplicate]

    return vec_duplicate, master

def averaged_word_vectorizer_glove(corpus, model, num_features):
    vocabulary = set(model.dictionary)
    if(any(isinstance(i, list) for i in corpus)):
        features = [average_word_vectors_glove(tokenized_sentence, model, vocabulary, num_features)
                      for tokenized_sentence in corpus]
        return np.array(features)
    else:
          features = average_word_vectors_glove(corpus, model, vocabulary, num_features)
    return np.array(features)

def average_word_vectors_glove(words, model, vocabulary, num_features):  
    feature_vector = np.zeros((num_features,),dtype="float64")
    nwords = 0.  

    for word in words:
        if word in vocabulary: 
            nwords = nwords + 1.
            feature_vector = np.add(feature_vector, model.word_vectors[model.dictionary[word]])

    if nwords:
        feature_vector = np.divide(feature_vector, nwords)
        
    return feature_vector

# Complete Data

Determining the top-n values for the Recall Rate @ k

In [27]:
#Import Duplicate Reports
test = pd.read_csv('firefox_duplicate_reports.csv')
test = test.drop(columns=['Unnamed: 0'])
test['Description']= test['Description'].fillna('').astype(str).map(preprocess)
test = test.rename(columns={'Bug ID':'Bug_ID'})

#To Decide the Iterations
number_of_samples = test.shape[0]

if(number_of_samples > 200):
  number_of_samples = 200

In [28]:
#Evaluating the performance by Recall Rate
def evaluate(vec_acc, t2, t1):
  sum = 0
  for i,num in enumerate(vec_acc):
      sum = sum + int(num)
  recall_rate = (sum/len(vec_acc))*100
  print("Recall Rate : {} %".format(recall_rate))
  print("Time : ", (t2-t1)/60, "min")

In [30]:
#For k=1
vec_acc_top1=[]
t1_top1 = time.time()

for i in range(number_of_samples):
    print(f'\rRunning sample {i}', end='')
    sample = test.Description[i] 
    n = 1
    max_cluster =sim_with_clusters_lda_topn(sample, n)
    v=[]

    for max in max_cluster:
        exec('cluster = topic_{}'.format(max)) 
        exec('model = glove{}'.format(max))                
        exec('sent = topic_{}.Description'.format(max))
        
        cluster = cluster.rename(columns={'Bug ID':'Bug_ID'})
        topn = 1  
        modal = 'single'                                 
        
        #This will return the Top-N predicted master reports
        max_sim = compare_topn(model, cluster, sent, sample, topn, modal)
        t2_top1 = time.time()

        #Comparing the predicted value to the ground truth
        for num in max_sim:
            if (cluster.Bug_ID[num] == test.Duplicate_Bug_Ids[i]):
                v.append("1")
            else:
                v.append("0")

    if(all(x==v[0] for x in v)):
        vec_acc_top1.append("0")
    else:
        vec_acc_top1.append("1")

Running sample 199

In [31]:
#For k=5
vec_acc_top5=[]
t1_top5 = time.time()

for i in range(number_of_samples):
    print(f'\rRunning sample {i}', end='')
    sample = test.Description[i] 
    n = 3
    max_cluster =sim_with_clusters_lda_topn(sample, n)
    v=[]

    for max in max_cluster:
        exec('cluster = topic_{}'.format(max)) 
        exec('model = glove{}'.format(max))                
        exec('sent = topic_{}.Description'.format(max))
        
        cluster = cluster.rename(columns={'Bug ID':'Bug_ID'})
        topn = 1 
        modal = 'single'                                 
        
        #This will return the Top-N predicted master reports
        max_sim = compare_topn(model, cluster, sent, sample, topn, modal)
        t2_top5 = time.time()

        #Comparing the predicted value to the ground truth
        for num in max_sim:
            if (cluster.Bug_ID[num] == test.Duplicate_Bug_Ids[i]):
                v.append("1")
            else:
                v.append("0")

    if(all(x==v[0] for x in v)):
        vec_acc_top5.append("0")
    else:
        vec_acc_top5.append("1")

Running sample 199

In [32]:
#For k=10
vec_acc_top10=[]
t1_top10 = time.time()

for i in range(number_of_samples):
    print(f'\rRunning sample {i}', end='')
    sample = test.Description[i] 
    n = 3
    max_cluster =sim_with_clusters_lda_topn(sample, n)
    v=[]

    for max in max_cluster:
        exec('cluster = topic_{}'.format(max)) 
        exec('model = glove{}'.format(max))                
        exec('sent = topic_{}.Description'.format(max))
        
        cluster = cluster.rename(columns={'Bug ID':'Bug_ID'})
        topn = 3  
        modal = 'single'                                 
        
        #This will return the Top-N predicted master reports
        max_sim = compare_topn(model, cluster, sent, sample, topn, modal)
        t2_top10 = time.time()

        #Comparing the predicted value to the ground truth
        for num in max_sim:
            if (cluster.Bug_ID[num] == test.Duplicate_Bug_Ids[i]):
                v.append("1")
            else:
                v.append("0")

    if(all(x==v[0] for x in v)):
        vec_acc_top10.append("0")
    else:
        vec_acc_top10.append("1")

Running sample 199

In [33]:
#For k=100
vec_acc_top100=[]
t1_top100 = time.time()

for i in range(number_of_samples):
    print(f'\rRunning sample {i}', end='')
    sample = test.Description[i] 
    n = 3
    max_cluster =sim_with_clusters_lda_topn(sample, n)
    v=[]

    for max in max_cluster:
        exec('cluster = topic_{}'.format(max)) 
        exec('model = glove{}'.format(max))                
        exec('sent = topic_{}.Description'.format(max))
        
        cluster = cluster.rename(columns={'Bug ID':'Bug_ID'})
        topn = 33   
        modal = 'single'                                 
        
        #This will return the Top-N predicted master reports
        max_sim = compare_topn(model, cluster, sent, sample, topn, modal)
        t2_top100 = time.time()

        #Comparing the predicted value to the ground truth
        for num in max_sim:
            if (cluster.Bug_ID[num] == test.Duplicate_Bug_Ids[i]):
                v.append("1")
            else:
                v.append("0")

    if(all(x==v[0] for x in v)):
        vec_acc_top100.append("0")
    else:
        vec_acc_top100.append("1")

Running sample 199

In [34]:
print('For k=1')
evaluate(vec_acc_top1, t2_top1, t1_top1)

print('\nFor k=5')
evaluate(vec_acc_top5, t2_top5, t1_top5)

print('\nFor k=10')
evaluate(vec_acc_top10, t2_top10, t1_top10)

print('\nFor k=100')
evaluate(vec_acc_top100, t2_top100, t1_top100)

For k=1
Recall Rate : 0.0 %
Time :  9.957866668701172 min

For k=5
Recall Rate : 5.0 %
Time :  21.93537183602651 min

For k=10
Recall Rate : 7.5 %
Time :  21.739986900488535 min

For k=100
Recall Rate : 13.0 %
Time :  21.631046350797018 min


# Textually Similar Data

Determining the top-n values for the Recall Rate @ k

In [35]:
#Import Textually Similar Data
test_sim = pd.read_csv('/content/drive/Shareddrives/DSCI-644-Team-5/PreProcessedData/Firefox_final_sim.csv')

#Cleaning the Data
test_sim = test_sim.drop(columns=['Unnamed: 0'])
test_sim['Description']= test_sim['Description'].fillna('').astype(str).map(preprocess)
test_sim.replace("", np.nan, inplace=True)
test_sim.dropna(subset = ["Duplicate_Bug_Ids"], inplace=True)
test_sim = test_sim.reset_index(drop=True)

#To Decide the Iterations
number_of_samples_sim = test_sim.shape[0]

if(number_of_samples_sim > 200):
  number_of_samples_sim = 200

In [36]:
#For k=1
vec_acc_top1_sim=[]
t1_top1_sim = time.time()

for i in range(number_of_samples_sim):
    print(f'\rRunning sample {i}', end='')
    sample = test_sim.Description[i] 
    n = 1
    max_cluster =sim_with_clusters_lda_topn(sample, n)
    v=[]

    for max in max_cluster:
        exec('cluster = topic_{}'.format(max)) 
        exec('model = glove{}'.format(max))                
        exec('sent = topic_{}.Description'.format(max))
      
        cluster = cluster.rename(columns={'Bug ID':'Bug_ID'})
        topn = 1   
        modal = 'single'                                 
        
        #This will return the Top-N predicted master reports
        max_sim = compare_topn(model, cluster, sent, sample, topn, modal)
        t2_top1_sim = time.time()

        #Comparing the predicted value to the ground truth
        for num in max_sim:
            if (cluster.Bug_ID[num] == test_sim.Duplicate_Bug_Ids[i]):
                v.append("1")
            else:
                v.append("0")

    if(all(x==v[0] for x in v)):
        vec_acc_top1_sim.append("0")
    else:
        vec_acc_top1_sim.append("1")

Running sample 199

In [37]:
#For k=5
vec_acc_top5_sim=[]
t1_top5_sim = time.time()

for i in range(number_of_samples_sim):
    print(f'\rRunning sample {i}', end='')
    sample = test_sim.Description[i]
    n = 3
    max_cluster =sim_with_clusters_lda_topn(sample, n)
    v=[]

    for max in max_cluster:
        exec('cluster = topic_{}'.format(max))             
        exec('model = glove{}'.format(max))                
        exec('sent = topic_{}.Description'.format(max))     
        
        cluster = cluster.rename(columns={'Bug ID':'Bug_ID'})
        topn = 1           
        modal = 'single'        
        
        #This will return the Top-N predicted master reports
        max_sim = compare_topn(model, cluster, sent, sample, topn, modal)
        t2_top5_sim = time.time()

        #Comparing the predicted value to the ground truth
        for num in max_sim:
            if (cluster.Bug_ID[num] == test_sim.Duplicate_Bug_Ids[i]):
                v.append("1")
            else:
                v.append("0")

    if(all(x==v[0] for x in v)):
        vec_acc_top5_sim.append("0")
    else:
        vec_acc_top5_sim.append("1")

Running sample 199

In [38]:
#For k=10
vec_acc_top10_sim=[]
t1_top10_sim = time.time()

for i in range(number_of_samples_sim):
    print(f'\rRunning sample {i}', end='')
    sample = test_sim.Description[i] 
    n = 3
    max_cluster =sim_with_clusters_lda_topn(sample, n)
    v=[]

    for max in max_cluster:
        exec('cluster = topic_{}'.format(max))              
        exec('model = glove{}'.format(max))               
        exec('sent = topic_{}.Description'.format(max))    
        
        cluster = cluster.rename(columns={'Bug ID':'Bug_ID'})
        topn = 3          
        modal = 'single'                                    
        
        #This will return the Top-N predicted master reports
        max_sim = compare_topn(model, cluster, sent, sample, topn, modal)
        t2_top10_sim = time.time()

        #Comparing the predicted value to the ground truth
        for num in max_sim:
            if (cluster.Bug_ID[num] == test_sim.Duplicate_Bug_Ids[i]):
                v.append("1")
            else:
                v.append("0")

    if(all(x==v[0] for x in v)):
        vec_acc_top10_sim.append("0")
    else:
        vec_acc_top10_sim.append("1")

Running sample 199

In [39]:
#For k=100
vec_acc_top100_sim=[]
t1_top100_sim = time.time()

for i in range(number_of_samples_sim):
    print(f'\rRunning sample {i}', end='')
    sample = test_sim.Description[i]
    n = 3
    max_cluster =sim_with_clusters_lda_topn(sample, n)
    v=[]

    for max in max_cluster:
        exec('cluster = topic_{}'.format(max))              
        exec('model = glove{}'.format(max))               
        exec('sent = topic_{}.Description'.format(max))     
        
        cluster = cluster.rename(columns={'Bug ID':'Bug_ID'})
        topn = 33          
        modal = 'single'    

        #This will return the Top-N predicted master reports
        max_sim = compare_topn(model, cluster, sent, sample, topn, modal)
        t2_top100_sim = time.time()

        #Comparing the predicted value to the ground truth
        for num in max_sim:
            if (cluster.Bug_ID[num] == test_sim.Duplicate_Bug_Ids[i]):
                v.append("1")
            else:
                v.append("0")

    if(all(x==v[0] for x in v)):
        vec_acc_top100_sim.append("0")
    else:
        vec_acc_top100_sim.append("1")

Running sample 199

In [40]:
print('For k=1')
evaluate(vec_acc_top1_sim, t2_top1_sim, t1_top1_sim)

print('\nFor k=5')
evaluate(vec_acc_top5_sim, t2_top5_sim, t1_top5_sim)

print('\nFor k=10')
evaluate(vec_acc_top10_sim, t2_top10_sim, t1_top10_sim)

print('\nFor k=100')
evaluate(vec_acc_top100_sim, t2_top100_sim, t1_top100_sim)

For k=1
Recall Rate : 0.0 %
Time :  14.386229968070984 min

For k=5
Recall Rate : 3.0 %
Time :  24.397793928782146 min

For k=10
Recall Rate : 3.5000000000000004 %
Time :  24.308109386761984 min

For k=100
Recall Rate : 6.0 %
Time :  24.257803535461427 min


# Textually Dissimilar Data

Determining the top-n values for the Recall Rate @ k

In [41]:
#Import Textually Similar Data
test_dissim = pd.read_csv('/content/drive/Shareddrives/DSCI-644-Team-5/PreProcessedData/Firefox_final_dis.csv')

#Cleaning the Data
test_dissim = test_dissim.drop(columns=['Unnamed: 0'])
test_dissim['Description']= test_dissim['Description'].fillna('').astype(str).map(preprocess)
test_dissim.replace("", np.nan, inplace=True)
test_dissim.dropna(subset = ["Duplicate_Bug_Ids"], inplace=True)
test_dissim = test_dissim.reset_index(drop=True)

#To Decide the Iterations
number_of_samples_dis = test_dissim.shape[0]

if(number_of_samples_dis > 200):
  number_of_samples_dis = 200

In [42]:
#For k=1
vec_acc_top1_dis=[]
t1_top1_dis = time.time()

for i in range(number_of_samples_dis):
    print(f'\rRunning sample {i}', end='')
    sample = test_dissim.Description[i] 
    n = 1
    max_cluster =sim_with_clusters_lda_topn(sample, n)
    v=[]
    
    for max in max_cluster:
        exec('cluster = topic_{}'.format(max))              
        exec('model = glove{}'.format(max))               
        exec('sent = topic_{}.Description'.format(max))     
        
        cluster = cluster.rename(columns={'Bug ID':'Bug_ID'})
        topn = 1              
        modal = 'single'                                    
        
        #This will return the Top-N predicted master reports
        max_sim = compare_topn(model, cluster, sent, sample, topn, modal)
        t2_top1_dis = time.time()

        #Comparing the predicted value to the ground truth
        for num in max_sim:
            if (cluster.Bug_ID[num] == test_dissim.Duplicate_Bug_Ids[i]):
                v.append("1")
            else:
                v.append("0")

    if(all(x==v[0] for x in v)):
        vec_acc_top1_dis.append("0")
    else:
        vec_acc_top1_dis.append("1")

Running sample 199

In [43]:
#For k=5
vec_acc_top5_dis=[]
t1_top5_dis = time.time()

for i in range(number_of_samples_dis):
    print(f'\rRunning sample {i}', end='')
    sample = test_dissim.Description[i] 
    n = 3
    max_cluster =sim_with_clusters_lda_topn(sample, n)
    v=[]

    for max in max_cluster:
        exec('cluster = topic_{}'.format(max))                       
        exec('model = glove{}'.format(max))                
        exec('sent = topic_{}.Description'.format(max))     
        
        cluster = cluster.rename(columns={'Bug ID':'Bug_ID'})
        topn = 1          
        modal = 'single'                                    
        
        #This will return the Top-N predicted master reports
        max_sim = compare_topn(model, cluster, sent, sample, topn, modal)
        t2_top5_dis = time.time()

        #Comparing the predicted value to the ground truth
        for num in max_sim:
            if (cluster.Bug_ID[num] == test_dissim.Duplicate_Bug_Ids[i]):
                v.append("1")
            else:
                v.append("0")

    if(all(x==v[0] for x in v)):
        vec_acc_top5_dis.append("0")
    else:
        vec_acc_top5_dis.append("1")

Running sample 199

In [44]:
#For k=10
vec_acc_top10_dis=[]
t1_top10_dis = time.time()

for i in range(number_of_samples_dis):
    print(f'\rRunning sample {i}', end='')
    sample = test_dissim.Description[i] 
    n = 3
    max_cluster =sim_with_clusters_lda_topn(sample, n)
    v=[]

    for max in max_cluster:
        exec('cluster = topic_{}'.format(max))                         
        exec('model = glove{}'.format(max))                
        exec('sent = topic_{}.Description'.format(max))     
        
        cluster = cluster.rename(columns={'Bug ID':'Bug_ID'})
        topn = 3      
        modal = 'single'                                  
        
        #This will return the Top-N predicted master reports
        max_sim = compare_topn(model, cluster, sent, sample, topn, modal)
        t2_top10_dis = time.time()

        #Comparing the predicted value to the ground truth
        for num in max_sim:
            if (cluster.Bug_ID[num] == test_dissim.Duplicate_Bug_Ids[i]):
                v.append("1")
            else:
                v.append("0")

    if(all(x==v[0] for x in v)):
        vec_acc_top10_dis.append("0")
    else:
        vec_acc_top10_dis.append("1")

Running sample 199

In [45]:
#For k=100
vec_acc_top100_dis=[]
t1_top100_dis = time.time()

for i in range(number_of_samples_dis):
    print(f'\rRunning sample {i}', end='')
    sample = test_dissim.Description[i] 
    n = 3
    max_cluster =sim_with_clusters_lda_topn(sample, n)
    v=[]

    for max in max_cluster:
        exec('cluster = topic_{}'.format(max))                
        exec('model = glove{}'.format(max))               
        exec('sent = topic_{}.Description'.format(max))    
        
        cluster = cluster.rename(columns={'Bug ID':'Bug_ID'})
        topn = 33         
        modal = 'single'                                   
        
        #This will return the Top-N predicted master reports
        max_sim = compare_topn(model, cluster, sent, sample, topn, modal)
        t2_top100_dis = time.time()

        #Comparing the predicted value to the ground truth
        for num in max_sim:
            if (cluster.Bug_ID[num] == test_dissim.Duplicate_Bug_Ids[i]):
                v.append("1")
            else:
                v.append("0")

    if(all(x==v[0] for x in v)):
        vec_acc_top100_dis.append("0")
    else:
        vec_acc_top100_dis.append("1")

Running sample 199

In [46]:
print('For k=1')
evaluate(vec_acc_top1_dis, t2_top1_dis, t1_top1_dis)

print('\nFor k=5')
evaluate(vec_acc_top5_dis, t2_top5_dis, t1_top5_dis)

print('\nFor k=10')
evaluate(vec_acc_top10_dis, t2_top10_dis, t1_top10_dis)

print('\nFor k=100')
evaluate(vec_acc_top100_dis, t2_top100_dis, t1_top100_dis)

For k=1
Recall Rate : 0.0 %
Time :  7.60915093421936 min

For k=5
Recall Rate : 6.0 %
Time :  20.23075507481893 min

For k=10
Recall Rate : 8.5 %
Time :  22.185565944512685 min

For k=100
Recall Rate : 14.499999999999998 %
Time :  22.22200423081716 min
